# Pipeline Execution
This file is a file to run the whole pipeline according to discussion on Dec. 12
<br>
Procedure:
1. Pre-process data: Only use TF that are in gold standard, as discussed in class, prepare scores df
2. For each model, predict GRN edges (with scores), using optimum hyperparameter, then standardize weights, compile each score to a 'score' table, with columns sources of score (including uniform scores from random clf) and index the edges
3. Visualize PRC and ROC for each method for full nv dataframe
4. Statistically examine each method by randomly selecting target edges, getting the mean auprc and stdev, conduct tests (vs random t test, anova, tukey if anove rejected)

## 1. Load data and Pre-process

In [ ]:
import pandas as pd
import grn_preds

# Read in expression, tf list, and gold files
nv_df = pd.read_csv('../dataset/yeast_networks/expression/NatVar.txt', sep='\t').set_index('Gene')
nv_tf = pd.read_csv('../dataset/yeast_networks/expression/NatVar_TF_names.txt', sep='\t', header=None)
nv_tf.columns = ['TF']

nv_gold = grn_preds.generate_gold_dataset(['../dataset/yeast_networks/gold/MacIsaac2.NatVar.txt', 
                                            '../dataset/yeast_networks/gold/YEASTRACT_Count3.NatVar.txt',
                                            '../dataset/yeast_networks/gold/YEASTRACT_Type2.NatVar.txt'])

# Filter tf list based on gold
nv_tf = nv_tf[nv_tf.loc[:, 'TF'].isin(set(nv_gold.loc[:, 'Regulator']))]

# Generate scores df
nv_scores_df = grn_preds.generate_possible_edges(nv_tf.loc[:, 'TF'], nv_df)
nv_scores_df = grn_preds.populate_actual_column(nv_scores_df, nv_gold)

## 2. Predict GRN edges with scores, using optimum hyperparameter or example hyperparameter, then store in compiled table

### 2a. Predict GRN edges with optimum hyperparameter (Takes about 10-12 hours, make sure back this file up!)

In [ ]:
'''
for target in nv_df.index:
    lasso_edges, lasso_scores = grn_preds.grn_lasso(target, nv_tf.loc[:, 'TF'], nv_df, alphas = [0.001, 0.01, 0.1, 1, 10, 100], cv = 5)
    rf_edges, rf_scores = grn_preds.grn_regforest(target, nv_tf.loc[:, 'TF'], nv_df, n_estimators = 100, max_depth = 8, bootstrap = True, min_samples_leaf = 10, n_jobs = -1)
    svr_edges, svr_scores = grn_preds.grn_linear_svr(target, nv_tf.loc[:, 'TF'], nv_df)
    
    nv_scores_df.loc[lasso_edges, 'Lasso scores'] = abs(lasso_scores)
    nv_scores_df.loc[rf_edges, 'Regforest scores'] = rf_scores
    nv_scores_df.loc[svr_edges, 'SVR scores'] = abs(svr_scores)
    
nv_scores_df.to_csv('../dataset/nv_scores_df.csv')
'''
1

### 2b. Standardize weights so it has mean 0 and standard deviation 1, then add each weight with `abs(min(weight))`

In [ ]:
'''
# Load nv_scores_df

nv_scores_df = pd.read_csv('../dataset/nv_scores_df.csv', index_col=0)
# Standardize scores for each scoring methods
methods = ["Lasso", "Regforest", "SVR"]
i = 0
print(len(set(nv_scores_df.loc[:, 'Target'])))
for method in methods:
    nv_scores_df.loc[:, f'Standardized {method} scores'] = 0

for target in list(set(nv_scores_df.loc[:, 'Target'])):
    if i % 10 == 0:
        print(i)
    for method in methods:
        new_index, standardized_scores = grn_preds.standardize_scores(target, nv_scores_df, method)
        nv_scores_df.loc[new_index, f'Standardized {method} scores'] = standardized_scores
    
    i += 1

nv_scores_df.to_csv('../dataset/nv_scores_df_stand.csv')
'''

In [ ]:
nv_scores_df_stand = pd.read_csv('../dataset/nv_scores_df_stand.csv', index_col=0)

In [ ]:
# Remove negative scores from weights in standardized scores by adding them with the smallest absolute value
nv_scores_df_stand.loc[:, 'Standardized Lasso scores'] = nv_scores_df_stand.loc[:, 'Standardized Lasso scores'] + abs(min(nv_scores_df_stand.loc[:, 'Standardized Lasso scores']))
nv_scores_df_stand.loc[:, 'Standardized Regforest scores'] = nv_scores_df_stand.loc[:, 'Standardized Regforest scores'] + abs(min(nv_scores_df_stand.loc[:, 'Standardized Regforest scores']))
nv_scores_df_stand.loc[:, 'Standardized SVR scores'] = nv_scores_df_stand.loc[:, 'Standardized SVR scores'] + abs(min(nv_scores_df_stand.loc[:, 'Standardized SVR scores']))

## 3. Visualize PRC and ROC for each method for full nv dataframe

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import auc, roc_curve, precision_recall_curve

# Get fpr, tpr stat for different method
roc_dic = {'raw': {}, 'stand': {}}
prc_dic = {'raw': {}, 'stand': {}}

for method in ['Lasso', 'Regforest', 'SVR']:
    # Prep labeling so code is easy to follow
    actual_edges = nv_scores_df_stand.loc[:, 'Actual']
    raw_label = f'{method} scores'
    stand_label = f'Standardized {method} scores'
    
    # Get ROC and PRC stats
    raw_roc = roc_curve(actual_edges, nv_scores_df_stand.loc[:, raw_label])
    stand_roc = roc_curve(actual_edges, nv_scores_df_stand.loc[:, stand_label])
    raw_prc = precision_recall_curve(actual_edges, nv_scores_df_stand.loc[:, raw_label])
    stand_prc = precision_recall_curve(actual_edges, nv_scores_df_stand.loc[:, stand_label])
    
    # Get AUC for each curve
    auc_raw_roc = auc(raw_roc[0], raw_roc[1])
    auc_stand_roc = auc(stand_roc[0], stand_roc[1])
    auc_raw_prc = auc(raw_prc[1], raw_prc[0])
    auc_stand_prc = auc(stand_prc[1], stand_prc[0])
    
    # Store to respective dictionary
    roc_dic['raw'][raw_label] = (raw_roc[0], raw_roc[1], auc_raw_roc)
    roc_dic['stand'][stand_label] = (stand_roc[0], stand_roc[1], auc_stand_roc)
    prc_dic['raw'][raw_label] = (raw_prc[1], raw_prc[0], auc_raw_prc)
    prc_dic['stand'][stand_label] = (stand_prc[1], stand_prc[0], auc_raw_prc)


# Get stats for uniform scores

## Get ROC and PRC stats
uniform_roc = roc_curve(actual_edges, nv_scores_df_stand.loc[:, 'Uniform score'])
uniform_prc = precision_recall_curve(actual_edges, nv_scores_df_stand.loc[:, 'Uniform score'])

## Get AUC for each curve
uniform_auc_roc = auc(uniform_roc[0], uniform_roc[1])
uniform_auc_prc = auc(uniform_prc[1], uniform_prc[0])

## Store to respective dictionary
roc_dic['uniform score'] = (uniform_roc[0], uniform_roc[1], uniform_auc_roc)
prc_dic['uniform score'] = (uniform_prc[1], uniform_prc[0], uniform_auc_prc)

In [ ]:
# Generate curves manually
raw_roc_fig = plt.figure(0, figsize=(10, 10))
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Figure 1: Raw ROC Curve')

colors = ['C1', 'C2', 'C3']
col_idx = 0
for method, stats in roc_dic['raw'].items():
    plt.plot(stats[0], stats[1], color = colors[col_idx], lw = 2, label = f'{method[:-7]}: {str(stats[2])[:6]}')
    col_idx += 1
col_idx = 0

plt.plot(roc_dic['uniform score'][0], roc_dic['uniform score'][1], color = 'black',
         lw = 2, linestyle = '--', label = f'Uniform score: {roc_dic["uniform score"][2]}')

plt.legend(loc = 'lower right')
plt.savefig('../figures/fig_1_raw_roc.png', format='png')
# plt.show()

In [ ]:
# Figure for Standardized ROC

stand_roc_fig = plt.figure(1, figsize=(10, 10))
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Figure 2: Standardized ROC Curve')

colors = ['C1', 'C2', 'C3']
col_idx = 0
for method, stats in roc_dic['stand'].items():
    plt.plot(stats[0], stats[1], color = colors[col_idx], lw = 2, label = f'{method[:-7]}: {str(stats[2])[:6]}')
    col_idx += 1
col_idx = 0

plt.plot(roc_dic['uniform score'][0], roc_dic['uniform score'][1], color = 'black',
         lw = 2, linestyle = '--', label = f'Uniform score: {roc_dic["uniform score"][2]}')

plt.legend(loc = 'lower right')
plt.savefig('../figures/fig_2_stand_roc.png', format='png')

# plt.show()

In [ ]:
# Figure for raw PRC
raw_prc_fig = plt.figure(1, figsize=(10, 10))
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Figure 3: Raw PRC Curve')

colors = ['C1', 'C2', 'C3']
col_idx = 0
for method, stats in prc_dic['raw'].items():
    plt.plot(stats[0], stats[1], color = colors[col_idx], lw = 2, label = f'{method[:-7]}: {str(stats[2])[:6]}')
    col_idx += 1
col_idx = 0

# plt.plot(prc_dic['uniform score'][0], prc_dic['uniform score'][1], color = 'black',
#          lw = 2, linestyle = '--', label = f'Uniform score: {prc_dic["uniform score"][2]}')

plt.legend(loc = 'upper right')
plt.savefig('../figures/fig_3_raw_prc.png', format='png')

# plt.show()

In [ ]:
# Figure for Standardized PRC

stand_prc_fig = plt.figure(1, figsize=(10, 10))
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Figure 4: Precision PRC Curve')

colors = ['C1', 'C2', 'C3']
col_idx = 0
for method, stats in prc_dic['stand'].items():
    plt.plot(stats[0], stats[1], color = colors[col_idx], lw = 2, label = f'{method[:-7]}: {str(stats[2])[:6]}')
    col_idx += 1
col_idx = 0

# plt.plot(prc_dic['uniform score'][0], prc_dic['uniform score'][1], color = 'black',
#          lw = 2, linestyle = '--', label = f'Uniform score: {prc_dic["uniform score"][2]}')

plt.legend(loc = 'upper right')
plt.savefig('../figures/fig_4_stand_prc.png', format='png')

# plt.show()

## 4. Statistically examine each method by randomly selecting target edges
Getting the mean auprc and stdev, conduct tests (vs random t test, anova, tukey if anova rejected)

In [ ]:
'''
# We select 100 target genes for each sampling, and we sample 100 times from the scores, then we get each method's auc for each sampling, for ROC and PRC.
import random

# Prepare score dictionary / list
methods = ['Lasso', 'Regforest', 'SVR', 'Standardized Lasso', 'Standardized Regforest', 'Standardized SVR']
score_dic = {'auroc': {}, 'auprc': {}}

for method in methods:
    score_dic['auroc'][method] = []
    score_dic['auprc'][method] = []

for i in range(100):
    print(i)
    sampled_targets = random.sample(list(set(nv_scores_df_stand.loc[:, 'Target'])), 100)
    subset_df = nv_scores_df_stand[nv_scores_df_stand.loc[:, 'Target'].isin(sampled_targets)]

    for method in methods:
        method_auroc = grn_preds.generate_auroc(subset_df.loc[:, 'Actual'], subset_df.loc[:, f'{method} scores'], show = False)
        method_auprc = grn_preds.generate_auprc(subset_df.loc[:, 'Actual'], subset_df.loc[:, f'{method} scores'], show = False)

        score_dic['auroc'][method].append(method_auroc)
        score_dic['auprc'][method].append(method_auprc)

# Store to csv file
auroc_df = pd.DataFrame.from_dict(score_dic['auroc'])
auprc_df = pd.DataFrame.from_dict(score_dic['auprc'])

auroc_df.to_csv('../dataset/auroc_sampling_scores.csv', index=False)
auprc_df.to_csv('../dataset/auprc_sampling_scores.csv', index=False)
'''

In [ ]:
# Load auroc and auprc sampling df
auroc_df = pd.read_csv('../dataset/auroc_sampling_scores.csv')
auprc_df = pd.read_csv('../dataset/auprc_sampling_scores.csv')